In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def sieve(n):
    """Implementation of the sieve of Eratosthenes utilizing NumPy arrays"""
    # prepare array of primes with first two positions as False
    primes = np.ones(n, dtype=np.bool_)
    primes[:2] = False

    # loop over primes and mark composite numbers
    for p in range(2, int(n ** 0.5) + 1):
        if primes[p]:
            primes[p * p: n : p] = False

    return primes.nonzero()[0]

# if file ./data/primes_2**32.csv does not exist, generate it
if not os.path.exists("./data/primes_2**32.csv"):
    print("generating primes for testing")
    test = sieve(2**32)

    # save to csv file
    print("saving primes")
    # create directory data if not exits
    if not os.path.exists("./data/"):
        os.makedirs("./data/")
    np.savetxt("./data/primes_2**32.csv", test, delimiter=",", fmt="%d")
    print("done")
else:
    print("test file already exists")

In [ ]:
# print hardware information
print("hardware information:")
print('CPU:')
print(os.popen("lscpu | grep name").read().split(":")[1].strip())
print('GPU:')
!lspci | grep -i 'NVIDIA' | awk -F': ' '{print $2}'

In [ ]:
# run benchmarks

# CPU GMP benchmark
!g++ -lgmp -o benchmark.out benchmark.cpp -L./.. -lmiller_rabin -Wl,-rpath,./..
!./benchmark.out
!rm benchmark.out

# C/C++ CUDA benchmark
!nvcc -O3 miller_rabin_cuda_32.cu
!./a.out
!rm a.out

# Python Numba CUDA benchmark
%run miller_rabin_cuda.py

In [ ]:
# load log and display bar chart
df = pd.read_csv('./data/log.csv')
display(df)

df.sort_values(by='time', ascending=False, inplace=True)
# Extract the data columns
implementations = df['implementation']
times = df['time']

# Configure plot
plt.figure(figsize=(12, 8))
plt.grid(axis='y', which='major', color='#CCCCCC', linestyle='--', linewidth=1, alpha=0.5)

# Set the labels and title
plt.xlabel('Implementation')
plt.ylabel('Time (seconds)')
plt.title('Time Comparison')


# Add the data to the plot
plt.bar(implementations, times)

# fill cpu with red
plt.bar(implementations[0], times[0], color='#CC000F')
# fill python with green
plt.bar(implementations[2], times[2], color='#00CC00')

# # save to png
plt.savefig('./data/results.png', format='png', dpi=600)

plt.show()